In [1]:
import tokenizers
import numpy as np
import torch

In [2]:
from tokenizers import BertWordPieceTokenizer

In [3]:
tokenizer = BertWordPieceTokenizer("bert-base-uncased-vocab.txt",
                                   lowercase=True,
                                   add_special_tokens=False,
                                   clean_text=True)

In [4]:
lines = []
textfile = '/home/tucker/data/imdb/aclImdb/train/unsup_all.txt'
#textfile = '/home/tucker/data/news/training-monolingual/news.2007.en.shuffled'
with open(textfile) as f:
  long_line = f.read().lower().replace('<br /><br />', ' ')

In [5]:
TOK_CHUNK_MAX = 58
TOK_CHUNK_MIN = 57
CHAR_CHUNK_MAX = 384
def GetExamplesFromSplitLine(line):
  token_ids = []
  tokens = line
  i = 0
  while i*TOK_CHUNK_MAX < len(tokens):
    potential = tokens[i*TOK_CHUNK_MAX: (i+1)*TOK_CHUNK_MAX]

    if len(potential)>=TOK_CHUNK_MIN:
      token_ids.append("[CLS] "+" ".join(potential))
    i+=1

  return token_ids
  

In [51]:
half = len(long_line)//5

In [48]:
long_line[half+5]

'i'

In [8]:
long_line = long_line[:half+5]

In [6]:
l=long_line.split()

In [7]:
del long_line

In [8]:
seq_chunks = []

t = GetExamplesFromSplitLine(l)
for seq in t:
  seq_chunks.append(seq)

In [9]:
torch.save(seq_chunks, "tmp.chunks")

In [10]:
seq_chunks = torch.load("tmp.chunks")

In [11]:
del l

In [12]:
seq_chunks[0]

'[CLS] i admit, the great majority of films released before say 1933 are just not for me. of the dozen or so "major" silents i have viewed, one i loved (the crowd), and two were very good (the last command and city lights, that latter chaplin circa 1931). so i was apprehensive about this one, and humor is often'

In [13]:
len(seq_chunks)

200091

In [14]:
encoded = tokenizer.encode_batch(seq_chunks)

In [15]:
encoded[0].tokens[:10]

['[CLS]',
 'i',
 'admit',
 ',',
 'the',
 'great',
 'majority',
 'of',
 'films',
 'released']

In [16]:
len(encoded)

200091

In [17]:
del seq_chunks

In [18]:
a=encoded[0]

In [19]:
token_len = 96
char_len = 384

In [21]:
idx_to_char_map = {v:k for k,v in torch.load('char_to_idx_map2.pt').items()}

In [26]:
zerochar = idx_to_char_map[0]

In [27]:
def GetTokOffsets(encoded):
  offsets = list([[x,y] for x,y in encoded.offsets if x<char_len and y<char_len])[:token_len]
  return offsets

In [28]:
char_to_idx_map  = torch.load('char_to_idx_map2.pt')

In [29]:
token_list = []
char_list = []
enc_char_list = []
offset_pos = []
for e in encoded:
  tokens = e.ids[:token_len]
  token_pad = token_len - len(tokens)
  tokens += [0]*token_pad

  chars = str(e.original_str).lower()[:char_len]
  char_pad = char_len - len(chars)
  chars += zerochar*char_pad

  enc = []
  for c in chars:
    enc.append(char_to_idx_map.get(c,0))
  
  tok_offset_idxs = GetTokOffsets(e)
  offset_pad = token_len - len(tok_offset_idxs)
  tok_offset_idxs += [[0,0]]*offset_pad
  
  token_list.append(tokens)
  char_list.append(chars)
  enc_char_list.append(enc)
  offset_pos.append(tok_offset_idxs)                        

In [30]:
of = offset_pos[503][34]
of

[137, 149]

In [31]:
char_list[503][of[0]:of[1]]

'entertaining'

In [32]:
data = {}
data['token_ids'] = torch.tensor(token_list).short()

In [21]:
ndata['chars'] = char_list

In [36]:
data['token_start_offsets'] = torch.tensor(offset_pos).short()

In [34]:
data['chars_encoded'] = torch.tensor(enc_char_list).char()

In [37]:
torch.save(data, 'imdb_tok96_char_encoded384charidx2.pt')

In [23]:
ds = torch.load('imdb_tok96_char384.pt')

In [73]:
enc = []
for c in '[mask]':
  enc.append(char_to_idx_map.get(c,0))

In [74]:
enc

[34, 52, 40, 58, 50, 36]

In [40]:
tok_lens = np.array([len(e.ids) for e in  encoded])

In [41]:
np.percentile(tok_lens, 98)

93.0

In [36]:
char_lens = np.array([len(e.original_str) for e in  encoded])

In [37]:
np.percentile(char_lens, 98)

381.0

In [38]:
char_lens.min()

197